# Mod 4 Project - Starter Notebook

This notebook has been provided to you so that you can make use of the following starter code to help with the trickier parts of preprocessing the Zillow dataset. 

The notebook contains a rough outline the general order you'll likely want to take in this project. You'll notice that most of the areas are left blank. This is so that it's more obvious exactly when you should make use of the starter code provided for preprocessing. 

**_NOTE:_** The number of empty cells are not meant to infer how much or how little code should be involved in any given step--we've just provided a few for your convenience. Add, delete, and change things around in this notebook as needed!

# Some Notes Before Starting

This project will be one of the more challenging projects you complete in this program. This is because working with Time Series data is a bit different than working with regular datasets. In order to make this a bit less frustrating and help you understand what you need to do (and when you need to do it), we'll quickly review the dataset formats that you'll encounter in this project. 

## Wide Format vs Long Format

If you take a look at the format of the data in `zillow_data.csv`, you'll notice that the actual Time Series values are stored as separate columns. Here's a sample: 

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/df_head.png'>

You'll notice that the first seven columns look like any other dataset you're used to working with. However, column 8 refers to the median housing sales values for April 1996, column 9 for May 1996, and so on. This This is called **_Wide Format_**, and it makes the dataframe intuitive and easy to read. However, there are problems with this format when it comes to actually learning from the data, because the data only makes sense if you know the name of the column that the data can be found it. Since column names are metadata, our algorithms will miss out on what dates each value is for. This means that before we pass this data to our ARIMA model, we'll need to reshape our dataset to **_Long Format_**. Reshaped into long format, the dataframe above would now look like:

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/melted1.png'>

There are now many more rows in this dataset--one for each unique time and zipcode combination in the data! Once our dataset is in this format, we'll be able to train an ARIMA model on it. The method used to convert from Wide to Long is `pd.melt()`, and it is common to refer to our dataset as 'melted' after the transition to denote that it is in long format. 

# Helper Functions Provided

Melting a dataset can be tricky if you've never done it before, so you'll see that we have provided a sample function, `melt_data()`, to help you with this step below. Also provided is:

* `get_datetimes()`, a function to deal with converting the column values for datetimes as a pandas series of datetime objects
* Some good parameters for matplotlib to help make your visualizations more readable. 

Good luck!


# Step 1: Load the Data/Filtering for Chosen Zipcodes

In [28]:
import pandas as pd

In [30]:
df1 = pd.read_csv('zillow_data.csv')
df1.head(5)

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [31]:
df1['RegionName'].value_counts()

55324    1
74561    1
73538    1
31546    1
82070    1
        ..
75182    1
55343    1
1450     1
73129    1
65536    1
Name: RegionName, Length: 14723, dtype: int64

__Comments__\
Region name is the zipcode.
RegionID not relevant to the analysis.

In [32]:
df = pd.read_csv('Zillow_data/zhvi_all-homes_zipcode.csv')
df.head(3)

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,233265.0,...,1248340.0,1234262.0,1229890.0,1226466.0,1208024.0,1182758.0,1150900.0,1134880.0,1120949.0,1112549.0
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,211748.0,...,494425.0,493485.0,492514.0,491726.0,491562.0,492618.0,494017.0,494766.0,494546.0,494435.0
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,245773.0,...,1161916.0,1153259.0,1156287.0,1175142.0,1193746.0,1205413.0,1203165.0,1209735.0,1211403.0,1212520.0


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30443 entries, 0 to 30442
Columns: 303 entries, RegionID to 2020-06-30
dtypes: float64(294), int64(3), object(6)
memory usage: 70.4+ MB


In [34]:
df.describe()

,RegionID,SizeRank,RegionName,1996-01-31,1996-02-29,1996-03-31,1996-04-30,1996-05-31,1996-06-30,1996-07-31,...,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30
count,30443.000000,30443.000000,30443.000000,1.327800e+04,1.352000e+04,1.355900e+04,1.357900e+04,1.366200e+04,1.369600e+04,1.371800e+04,...,3.044300e+04,3.044300e+04,3.044300e+04,3.044300e+04,3.044300e+04,3.044300e+04,3.044300e+04,3.044300e+04,3.044300e+04,3.044300e+04
mean,80445.025687,15968.086424,48821.634202,1.249792e+05,1.246061e+05,1.245439e+05,1.245732e+05,1.246324e+05,1.246580e+05,1.247923e+05,...,2.246035e+05,2.252417e+05,2.258022e+05,2.263989e+05,2.270490e+05,2.277571e+05,2.284653e+05,2.290863e+05,2.295599e+05,2.300180e+05
std,25570.636929,9592.659553,27395.589200,8.256812e+04,8.235609e+04,8.226986e+04,8.205293e+04,8.189547e+04,8.175067e+04,8.163481e+04,...,2.287857e+05,2.291534e+05,2.295232e+05,2.301052e+05,2.308971e+05,2.318501e+05,2.327255e+05,2.332001e+05,2.331115e+05,2.330233e+05
min,58001.000000,0.000000,501.000000,1.108500e+04,1.106800e+04,1.108600e+04,1.108100e+04,1.109100e+04,1.100600e+04,1.088300e+04,...,1.211500e+04,1.217400e+04,1.230300e+04,1.248800e+04,1.268000e+04,1.281200e+04,1.274000e+04,1.275800e+04,1.267500e+04,1.259600e+04
25%,68866.500000,7694.000000,25984.500000,7.210100e+04,7.201850e+04,7.199800e+04,7.210050e+04,7.200075e+04,7.196700e+04,7.217025e+04,...,1.052645e+05,1.056320e+05,1.059335e+05,1.062160e+05,1.064970e+05,1.067270e+05,1.070195e+05,1.071660e+05,1.072890e+05,1.076295e+05
50%,79205.000000,15500.000000,48367.000000,1.071430e+05,1.066410e+05,1.066140e+05,1.067160e+05,1.068920e+05,1.068505e+05,1.070275e+05,...,1.634180e+05,1.640760e+05,1.645880e+05,1.650920e+05,1.655220e+05,1.662070e+05,1.667110e+05,1.671500e+05,1.676460e+05,1.681070e+05
75%,89229.500000,24035.000000,71675.500000,1.557038e+05,1.553472e+05,1.553485e+05,1.554900e+05,1.556012e+05,1.557130e+05,1.558642e+05,...,2.630395e+05,2.638305e+05,2.645315e+05,2.653320e+05,2.660165e+05,2.667940e+05,2.676945e+05,2.684140e+05,2.692260e+05,2.699960e+05
max,753844.000000,35187.000000,99929.000000,1.955400e+06,1.950233e+06,1.929140e+06,1.896476e+06,1.843693e+06,1.790852e+06,1.737201e+06,...,6.284529e+06,6.291254e+06,6.314141e+06,6.346305e+06,6.407222e+06,6.459116e+06,6.518578e+06,6.515411e+06,6.456533e+06,6.399640e+06


In [35]:
df.dtypes.unique()

array([dtype('int64'), dtype('O'), dtype('float64')], dtype=object)

In [36]:
#rename RegionName to ZipCode
df.rename(columns={'RegionName': 'zipcode'}, inplace=True)
#drop RegionID
df.drop(columns=['RegionID', 'RegionType', 'StateName'], inplace=True)
df.head(2)

,SizeRank,zipcode,State,City,Metro,CountyName,1996-01-31,1996-02-29,1996-03-31,1996-04-30,...,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30
0,0,10025,NY,New York,New York-Newark-Jersey City,New York County,233265.0,231813.0,231891.0,231866.0,...,1248340.0,1234262.0,1229890.0,1226466.0,1208024.0,1182758.0,1150900.0,1134880.0,1120949.0,1112549.0
1,1,60657,IL,Chicago,Chicago-Naperville-Elgin,Cook County,211748.0,211591.0,211180.0,211044.0,...,494425.0,493485.0,492514.0,491726.0,491562.0,492618.0,494017.0,494766.0,494546.0,494435.0


# Step 2: Data Preprocessing

In [38]:
def get_datetimes(df):
    return pd.to_datetime(df.columns.values[1:], format='%Y-%m')

In [39]:
get_datetimes(df)

ValueError: time data zipcode doesn't match format specified

# Step 3: EDA and Visualization

In [ ]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

# Step 4: Reshape from Wide to Long Format

In [ ]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['RegionName', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

# Step 5: ARIMA Modeling

# Step 6: Interpreting Results